In [1]:
# importing libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
warnings.filterwarnings('ignore')
import os

In [2]:
# loading the data

for _, dirnames, filenames in os.walk('C:\\Users\\ajaym\\Desktop\\ML_projects\\student_performance_indicator_project\\Data'):
    for filename in filenames:
        print(filename)

archive.zip
EDA.ipynb
Model_training.ipynb
Raw_data.csv
Untitled.ipynb
EDA-checkpoint.ipynb
Model_training-checkpoint.ipynb
Untitled-checkpoint.ipynb


In [3]:
mydata = pd.read_csv('Raw_data.csv')
mydata.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [4]:
# extracting X and Y variables

X = mydata.drop(columns = ['math score'], axis = 1)

X.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [5]:
y = mydata[['math score']]

y.head()

,math score
0,72
1,69
2,90
3,47
4,76


In [6]:
print('the number of categories in gender_column are :', format(mydata.gender.unique()))
print('the number of categories in race/ethnicity are :', format(mydata['race/ethnicity'].unique()))
print('the number of categories in parental level of education are :', format(mydata['parental level of education'].unique()))
print('the number of categories in lunch are :', format(mydata['lunch'].unique()))

the number of categories in gender_column are : ['female' 'male']
the number of categories in race/ethnicity are : ['group B' 'group C' 'group A' 'group D' 'group E']
the number of categories in parental level of education are : ["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']
the number of categories in lunch are : ['standard' 'free/reduced']


In [7]:
numeric_features = [feature for feature in X.columns if X[feature].dtype != 'O']

categoric_features = [feature for feature in mydata.columns if mydata[feature].dtype == 'O']

In [8]:
numeric_features

['reading score', 'writing score']

In [9]:
categoric_features

['gender',
 'race/ethnicity',
 'parental level of education',
 'lunch',
 'test preparation course']

In [10]:
# transforming the categorical data

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

In [11]:
numeric_transformer = StandardScaler()
cat_transformer = OneHotEncoder()

In [12]:
preprocessor = ColumnTransformer(
transformers=[
    ('OneHotEncoder', cat_transformer, categoric_features),
    ('StandardScaler', numeric_transformer, numeric_features)
])

In [13]:
preprocessor

ColumnTransformer(transformers=[('OneHotEncoder', OneHotEncoder(),
                                 ['gender', 'race/ethnicity',
                                  'parental level of education', 'lunch',
                                  'test preparation course']),
                                ('StandardScaler', StandardScaler(),
                                 ['reading score', 'writing score'])])

In [14]:
X.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [15]:
x_copy = X.copy()

In [16]:
x_copy.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [17]:
X = preprocessor.fit_transform(X)

In [18]:
transformedData = pd.DataFrame(X)

In [19]:
transformedData

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.193999,0.391492
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.427476,1.313269
2,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.770109,1.642475
3,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,-0.833899,-1.583744
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.605158,0.457333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,2.044215,1.774157
996,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,-0.970952,-0.859491
997,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.125472,-0.201079
998,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.605158,0.589015


In [20]:
# separating dataset into training and testing

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(transformedData, y, test_size = 0.2, random_state = 42)

In [21]:
X_train.shape , y_train.shape , X_test.shape , y_test.shape

((800, 19), (800, 1), (200, 19), (200, 1))

In [22]:
# creating a function with all evaluation metrics to check the performance of the model after training

def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, mse, rmse, r2_square

In [23]:
# my_models

In [24]:
models = {
    "Linear_Regression" : LinearRegression(),
    "Lasso" : Lasso(),
    "Ridge" : Ridge(),
    "K-Neighbors Regressor" : KNeighborsRegressor(),
    "Decision Tree" : DecisionTreeRegressor(),
    "Random Forest Regressor" : RandomForestRegressor(),
    "XGB Regressor" : XGBRegressor(),
    "CatBoost Regressor" : CatBoostRegressor(verbose = False),
    "AdaBoost Regressor" : AdaBoostRegressor()
}

model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    # training the model
    model.fit(X_train, y_train)

    
    # predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)


    # evaluating the model
    model_train_mae, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])


    print('performance of the model in training dataset')
    print('Mean absolute error {.4f}', format(model_train_mae))
    print('Root Mean Squared Error {.4f}', format(model_train_rmse))
    print('R2_score {.4f}', format(model_train_r2))

    print('---------------------------------------------------------------')


    print('performance of the model in testing dataset')
    print('Mean absolute error {.4f}', format(model_test_mae))
    print('Root Mean Squared Error {.4f}', format(model_test_rmse))
    print('R2_score {.4f}', format(model_test_r2))
    r2_list.append(model_test_r2)


    print('='*40)
    print('\n')

    

ValueError: too many values to unpack (expected 3)